In [ ]:
import pandas as pd
import xarray as xr

import config
from utils import processing

In [ ]:
df = pd.read_csv(config.RAW_STORM_DB_PATH)
df.head()

In [ ]:
df = processing.rename_columns(df, column_map=config.COL_RENAME_MAP)

In [ ]:
# load geopotential data and convert to height
geop, height = processing.load_geop_and_calc_elevation()

# load subgrid orography angle data
anor = xr.open_dataset(config.DATA_DIR / "std" / "anor.nc")

df = processing.get_orography_features(df, geop, height, anor)

In [ ]:
# select only the columns that are in the config
df = df[[col for col in config.FEATURE_COL_NAMES if col in df.columns]]

In [ ]:
df.to_csv(config.PROCESSED_DATASET_PATH, index=False)